# Example 1 - Default Approach

This notebook demonstrates how to use the TX2 dashboard with a sequence classification transformer using the default approach as described in the "Basic Usage" docs.

In [1]:
%cd -q ..
%load_ext autoreload
%autoreload 2
%matplotlib inline

We enable logging to view the output from `wrapper.prepare()` further down in the notebook. (It's a long running function, and logs which step it's on.)

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
import numpy as np
import torch
from tqdm.notebook import tqdm
# import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, BertTokenizer
import pandas as pd

In this example notebook, we use the 20 newsgroups dataset, which can be downloaded through sklearn via below:

In [4]:
from sklearn.datasets import fetch_20newsgroups

train_data = fetch_20newsgroups(subset='train')
test_data = fetch_20newsgroups(subset='test')

Defined below is a simple sequence classification model with a variable for the language model itself `l1`. Since it is a BERT model, we take the sequence embedding from the `[CLS]` token (via `output_1[0][:, 0, :])`) and pipe that into the linear layer.

In [5]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained("bert-base-cased")
        self.l2 = torch.nn.Linear(768, 20)

    def forward(self, ids, mask):
        output_1= self.l1(ids, mask)
        output = self.l2(output_1[0][:, 0, :]) # use just the [CLS] output embedding
        return output
    
model = BERTClass()
model.to("cuda")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Some simplistic data cleaning, and putting all data into dataframes for the wrapper

In [6]:
def clean_text(text):
    text = text[text.index("\n\n")+2:]
    text = text.replace("\n", " ")
    text = text.replace("    ", " ")
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.strip()
    return text

In [7]:
train_rows = []
for i in range(len(train_data["data"])):
    row = {}
    row["text"] = clean_text(train_data["data"][i])
    row["target"] = train_data['target'][i]
    if row["text"] == "" or row["text"] == " ": continue
    train_rows.append(row)
train_df = pd.DataFrame(train_rows)

In [8]:
test_rows = []
for i in range(len(test_data["data"])):
    row = {}
    row["text"] = clean_text(test_data["data"][i])
    row["target"] = test_data['target'][i]
    if row["text"] == "" or row["text"] == " ": continue
    test_rows.append(row)
test_df = pd.DataFrame(test_rows)

## Training

This section minimally trains the classification and language model - nothing fancy here, just to give the dashboard demo something to work with. Most of this is similar to the huggingface tutorial notebooks.

In [9]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-05)

In [10]:
class EncodedSet(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        print(self.len)

    def __getitem__(self, index):
        text = str(self.data.text[index])
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.target[index], dtype=torch.long)
        }

    def __len__(self):
        return self.len
    
train_set = EncodedSet(train_df, tokenizer, 256)
test_set = EncodedSet(test_df[:1000], tokenizer, 256)

train_params = {'batch_size': 16,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 2,
                'shuffle': True,
                'num_workers': 0
                }

# put everything into data loaders
train_loader = DataLoader(train_set, **train_params)
test_loader = DataLoader(test_set, **test_params)

11296
1000


In [11]:
device = "cuda"
def train(epoch):
    model.train()

    
    loss_history = []
    for _,data in tqdm(enumerate(train_loader, 0), total=len(train_loader), desc=f"Epoch {epoch}"):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask).squeeze()

        optimizer.zero_grad()
        loss = loss_function(outputs, targets)
        if _%100==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        loss_history.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         torch.cuda.empty_cache()
    return loss_history

In [12]:
losses = []
for epoch in range(1):
    losses.extend(train(epoch))

Epoch 0:   0%|          | 0/706 [00:00<?, ?it/s]

/home/81n/miniconda3/envs/adapd_nlp_hf/lib/python3.8/site-packages/transformers-4.1.1-py3.8.egg/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).


Epoch: 0, Loss:  3.0427584648132324
Epoch: 0, Loss:  1.9246082305908203
Epoch: 0, Loss:  1.3301795721054077
Epoch: 0, Loss:  0.6518872976303101
Epoch: 0, Loss:  0.8348802924156189
Epoch: 0, Loss:  0.7776210308074951
Epoch: 0, Loss:  0.3987298011779785
Epoch: 0, Loss:  0.27758270502090454


The wrapper uses an `encodings` dictionary for various labels/visualizations, and can be set up with something similar to:

In [13]:
encodings = {}
for index, entry in enumerate(train_data["target_names"]):
    encodings[entry] = index
encodings

{'alt.atheism': 0,
 'comp.graphics': 1,
 'comp.os.ms-windows.misc': 2,
 'comp.sys.ibm.pc.hardware': 3,
 'comp.sys.mac.hardware': 4,
 'comp.windows.x': 5,
 'misc.forsale': 6,
 'rec.autos': 7,
 'rec.motorcycles': 8,
 'rec.sport.baseball': 9,
 'rec.sport.hockey': 10,
 'sci.crypt': 11,
 'sci.electronics': 12,
 'sci.med': 13,
 'sci.space': 14,
 'soc.religion.christian': 15,
 'talk.politics.guns': 16,
 'talk.politics.mideast': 17,
 'talk.politics.misc': 18,
 'talk.religion.misc': 19}

## TX2

This section shows how to put everything into the TX2 wrapper to get the dashboard widget displayed.

In [14]:
from tx2.wrapper import Wrapper
from tx2.dashboard import Dashboard

In [17]:
wrapper = Wrapper(
    train_df=train_df, 
    test_df=test_df[:2000], 
    encodings=encodings, 
    input_col_name="text", 
    target_col_name="target", 
    classifier=model, 
    language_model=model.l1, 
    tokenizer=tokenizer,
    overwrite=False
)
wrapper.prepare()

INFO:root:Checking for cached predictions...
INFO:root:cached version 'data/predictions.json' found
INFO:root:Checking for cached embeddings...
INFO:root:cached version 'data/embedding_training.json' found
INFO:root:cached version 'data/embedding_testing.json' found
INFO:root:Checking for cached projections...
INFO:root:cached version 'data/projections_training.json' found
INFO:root:cached version 'data/projections_testing.json' found
INFO:root:cached version 'data/projector.pkl.gz' found
INFO:root:Checking for cached salience maps...
INFO:root:cached version 'data/salience.pkl.gz' found
INFO:root:Checking for cached cluster profiles...
INFO:root:cached version 'data/cluster_profiles.pkl.gz' found
INFO:root:cached version 'data/clusters.json' found
INFO:root:cached version 'data/cluster_labels.json' found
INFO:root:cached version 'data/cluster_words.json' found
INFO:root:cached version 'data/cluster_class_words.json' found


In [18]:
%matplotlib agg
dash = Dashboard(wrapper)
dash.render()

/home/81n/miniconda3/envs/adapd_nlp_hf/lib/python3.8/site-packages/transformers-4.1.1-py3.8.egg/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).


To play with different UMAP and DBSCAN arguments without having to recompute the entire `prepare()` function, you can use `recompute_projections` (which will recompute both the projections and visual clusterings) or `recompute_visual_clusterings` (which will only redo the clustering)

In [19]:
# wrapper.recompute_visual_clusterings(dbscan_args=dict())
# wrapper.recompute_projections(umap_args=dict(min_dist=.2), dbscan_args=dict())

To test or debug the classification model/see what raw outputs the viusualizations are getting, or create your own visualization tools, you can manually call the `classify()`, `soft_classify()`, `embed()` functions, or get access to any of the cached data as seen in the bottom cell

In [20]:
wrapper.classify(["testing"])

[7]

In [21]:
wrapper.soft_classify(["testing"])

[[-0.9905167818069458,
  0.02280207723379135,
  -0.7104247808456421,
  -0.1466187685728073,
  0.014081034809350967,
  -0.44134095311164856,
  1.3942747116088867,
  2.6510210037231445,
  2.26328444480896,
  -0.7248047590255737,
  -0.18305812776088715,
  -1.2000950574874878,
  0.41281259059906006,
  0.07312041521072388,
  0.10818176716566086,
  -0.4958801567554474,
  -0.7442573308944702,
  -0.7798997759819031,
  -1.0418503284454346,
  -0.6585859060287476]]

In [22]:
wrapper.embed(["testing"])

[[0.24071310460567474,
  -0.17693574726581573,
  1.1926298141479492,
  -0.21286188066005707,
  -0.5541257858276367,
  -0.11262716352939606,
  0.380913108587265,
  -0.8688269257545471,
  0.6813204884529114,
  -1.372365951538086,
  0.06777114421129227,
  0.5549997687339783,
  0.722969114780426,
  0.8163159489631653,
  -0.46859559416770935,
  -0.04362483695149422,
  0.38890770077705383,
  -0.8300358057022095,
  1.0483708381652832,
  0.7251316905021667,
  0.9786590337753296,
  -0.7416961193084717,
  0.25983184576034546,
  -0.7327815890312195,
  -1.0317604541778564,
  -0.2332402616739273,
  0.09239533543586731,
  0.6674163937568665,
  0.9412500262260437,
  -1.6647350788116455,
  -0.301195353269577,
  0.284109503030777,
  0.3736993670463562,
  1.1517614126205444,
  0.22863119840621948,
  -0.08315934985876083,
  0.5433588027954102,
  0.3460302948951721,
  0.1866254210472107,
  1.1284598112106323,
  0.0030066806357353926,
  -0.42831340432167053,
  -0.5717208385467529,
  -0.3800235688686371,
  

In [ ]:
# cached data:
# wrapper.embeddings_training
# wrapper.embeddings_testing
# wrapper.projector
# wrapper.projections_training
# wrapper.projections_testing
# wrapper.salience_maps
# wrapper.clusters
# wrapper.cluster_profiles
# wrapper.cluster_words_freqs
# wrapper.cluster_class_word_sets